In [14]:
!pip3 install gensim
!pip3 install pyLDAvis
!pip3 install numpy
!pip3 install wordcloud

import pandas as pd
import matplotlib.pyplot as plt
from gensim import corpora, models
from gensim.similarities import MatrixSimilarity
from gensim.utils import SaveLoad
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from re import sub
import pyLDAvis
from collections import Counter
from gensim.matutils import corpus2csc, sparse2full, corpus2dense
from wordcloud import WordCloud
from sklearn.utils import resample
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
import seaborn as sns
import pyLDAvis.gensim_models
import numpy as np
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/ermi/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [10]:
df = pd.read_csv('cleaned_tweet_data.csv')

In [11]:
df.head()

,created_at,source,original_text,polarity,subjectivity,lang,favorite_count,retweet_count,original_author,followers_count,friends_count,possibly_sensitive,hashtags,user_mentions,place
0,2022-04-22 22:17:05+00:00,"<a href=""http://twitter.com/download/android"" ...",RT @NorthstarCharts: The 10-year yield is tell...,0.16,0.540000,en,188.0,43.0,davideiacovozzi,18,18,NaN,"[{'text': 'gold', 'indices': [116, 121]}, {'te...","[{'screen_name': 'NorthstarCharts', 'name': 'N...",NaN
1,2022-04-22 13:44:53+00:00,"<a href=""http://twitter.com/download/android"" ...",RT @MichaelAArouet: German 10y mortgage rate w...,NaN,0.175000,en,NaN,NaN,davideiacovozzi,18,18,NaN,[],"[{'screen_name': 'MichaelAArouet', 'name': 'Mi...",NaN
2,2022-04-22 06:10:34+00:00,"<a href=""http://twitter.com/download/android"" ...",RT @goldseek: When? https://t.co/kO2FfHKaZg,0.00,0.000000,en,193.0,26.0,davideiacovozzi,18,18,False,[],"[{'screen_name': 'goldseek', 'name': 'Peter ⚒ ...",NaN
3,2022-04-21 17:22:09+00:00,"<a href=""http://twitter.com/download/android"" ...",RT @charliebilello: The 30-year mortgage rate ...,0.00,0.183333,en,620.0,213.0,davideiacovozzi,18,18,NaN,[],"[{'screen_name': 'charliebilello', 'name': 'Ch...",NaN
4,2022-04-21 10:32:26+00:00,"<a href=""http://twitter.com/download/android"" ...",RT @biancoresearch: Rates rise until something...,-0.40,0.400000,en,1787.0,417.0,davideiacovozzi,18,18,False,[],"[{'screen_name': 'biancoresearch', 'name': 'Ji...",NaN


In [12]:
df.shape

(16367, 15)

In [15]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

df['original_text'][0]

# convert all original_text fields to strings
df['original_text'] = df['original_text'].astype(str)
df['clean_text'] = df['original_text'].str.replace('[^\w\s]','')
df['clean_text'] = df['clean_text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
df['clean_text'] = df['clean_text'].apply(lambda x: " ".join(x.lower() for x in x.split()))
# re.sub('\w*\d\w*', '', text)
df['clean_text'] = df['clean_text'].str.replace('\w*\d\w*','')
# df.head()


# normally we tokenize using words so each word as a token
#Lines 4 to 6
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
df['clean_text'] = df['clean_text'].apply(lambda x: " ".join([stemmer.stem(word) for word in x.split()]))

df[['polarity','original_text','clean_text']].head()

print('hey')

<>:8: DeprecationWarning: invalid escape sequence \w
<>:12: DeprecationWarning: invalid escape sequence \w
<>:8: DeprecationWarning: invalid escape sequence \w
<>:12: DeprecationWarning: invalid escape sequence \w
/tmp/ipykernel_52727/620628855.py:8: DeprecationWarning: invalid escape sequence \w
  df['clean_text'] = df['original_text'].str.replace('[^\w\s]','')
/tmp/ipykernel_52727/620628855.py:12: DeprecationWarning: invalid escape sequence \w
  df['clean_text'] = df['clean_text'].str.replace('\w*\d\w*','')
/tmp/ipykernel_52727/620628855.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  df['clean_text'] = df['original_text'].str.replace('[^\w\s]','')
/tmp/ipykernel_52727/620628855.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  df['clean_text'] = df['clean_text'].str.replace('\w*\d\w*','')


hey


In [16]:
df['original_text'][0]

"RT @NorthstarCharts: The 10-year yield is telling us that there's a high risk of something 'breaking' in the system #gold #silver #crypto #…"

In [ ]:
df['clean_text'][0]

In [17]:
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /home/ermi/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /home/ermi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [18]:
#Data cleaning
wnl = WordNetLemmatizer()

def removePunc(myWord):
    """Function to remove punctuation from string inputs"""
    if myWord is None:
        return myWord
    else:
        return sub('[.:;()/!&-*@$,?^\d+]','',myWord)
        
def removeAscii(myWord):
    """Function to remove ascii from string input"""
    if myWord is None:
        return myWord
    else:
        return str(sub(r'[^\x00-\x7F]+','', myWord.strip()))

def lemmatize(myWord):
    """Function to lemmatize words"""
    if myWord is None:
        return myWord
    else:
        return str(wnl.lemmatize(myWord))

def removeStopWords(myWord):
    """Function to remove stop words"""
    if myWord is None:
        return myWord
    if myWord not in str(stopwords.words('english')):
        return myWord

def removeLinkUser(myWord):
    """Function to remove web addresses and twitter handles"""
    if not myWord.startswith('@') and not myWord.startswith('http'):
        return myWord

def prepText(myWord):
    """Final text pre-processing function"""
    return removeStopWords(
        lemmatize(
            removeAscii(
                removePunc(
                    removeLinkUser(
                        myWord.lower()
                    )
                )
            )
        )
    )

<>:9: DeprecationWarning: invalid escape sequence \d
<>:9: DeprecationWarning: invalid escape sequence \d
/tmp/ipykernel_52727/2831030072.py:9: DeprecationWarning: invalid escape sequence \d
  return sub('[.:;()/!&-*@$,?^\d+]','',myWord)


In [22]:
#Filtering Words
def filterTweetList(tweetList):
    """Remove stop words, lemmatize, and clean all tweets"""
    return [[prepText(word) for word
                in tweet.split()
                    if prepText(word) is not None]
                for tweet in tweetList]

In [23]:
cleanKagTweetList = filterTweetList(df['clean_text'])

In [24]:
cleanKagTweetList[:5]

[['northstarchart',
  'yield',
  'tell',
  'high',
  'risk',
  'someth',
  'break',
  'system',
  'gold',
  'silver',
  'crypto'],
 ['michaelaarouet',
  'german',
  'mortgag',
  'rate',
  'went',
  'hear',
  'sound',
  'german',
  'real',
  'estat',
  'bubbl',
  'burst'],
 ['goldseek'],
 ['charliebilello',
  'mortgag',
  'rate',
  'rise',
  'highest',
  'level',
  'sinc',
  'last',
  'year',
  'hit',
  'alltim'],
 ['biancoresearch',
  'rate',
  'rise',
  'someth',
  'break',
  'anyth',
  'broken',
  'yet']]

In [26]:
def makeDict(myTweetList):
    """Create dictionary from list of tokenized documents"""
    return corpora.Dictionary(myTweetList)

def makeCorpus(myTweetList,myDict):
    """Create corpus from list of tokenized documents"""
    return [myDict.doc2bow(tweet) for tweet in myTweetList]

def createLDA(myCorpus, myDictionary,myTopics=50,myPasses=10,myIterations=50,myAlpha=0.001):
    """LDA model call function"""
    return models.LdaMulticore(myCorpus, id2word=myDictionary, num_topics=myTopics, passes=myPasses,
    iterations=myIterations,alpha=myAlpha)

In [27]:
"""Create model objects"""
kagDict   = makeDict(cleanKagTweetList)
kagCorpus = makeCorpus(cleanKagTweetList, kagDict)
kagLda = createLDA(kagCorpus, kagDict)

"""Save model objects"""
SaveLoad.save(kagLda,'kaggleLDAmodel')
corpora.MmCorpus.serialize('kaggleCorpus.mm', kagCorpus)
kagDict.save('kaggleDictionary.dict')

In [28]:
kagLda = SaveLoad.load('kaggleLDAmodel')
kagDict = corpora.Dictionary.load('kaggleDictionary.dict')
kagCorpus = corpora.MmCorpus('kaggleCorpus.mm')

In [29]:
pyLDAvis.enable_notebook()
ldaViz = pyLDAvis.gensim_models.prepare(kagLda, kagCorpus, kagDict)

/home/ermi/anaconda3/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(
/home/ermi/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/ermi/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/ermi/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/ermi/anaconda3/lib/python3.9/site-packages/past/builtins/mis

In [30]:
kagLda.print_topics()

[(5,
  '0.031*"pm" + 0.025*"tomorrow" + 0.023*"wordl" + 0.017*"borisjohnson" + 0.017*"india" + 0.011*"narendramodi" + 0.011*"effort" + 0.011*"one" + 0.010*"make" + 0.010*"problem"'),
 (45,
  '0.015*"lol" + 0.012*"amp" + 0.010*"woman" + 0.008*"men" + 0.007*"democraci" + 0.007*"near" + 0.007*"call" + 0.007*"funni" + 0.007*"isra" + 0.007*"democrat"'),
 (14,
  '0.074*"price" + 0.029*"inflat" + 0.022*"chang" + 0.017*"oil" + 0.016*"increas" + 0.015*"gold" + 0.015*"fuel" + 0.015*"hike" + 0.015*"high" + 0.012*"valu"'),
 (27,
  '0.057*"srilanka" + 0.041*"protest" + 0.039*"lka" + 0.032*"cartoon" + 0.027*"economiccrisislk" + 0.022*"srilankacrisi" + 0.018*"tamil" + 0.016*"srilankaprotest" + 0.014*"gohomegota" + 0.012*"day"'),
 (16,
  '0.036*"biden" + 0.026*"inflat" + 0.020*"dollar" + 0.016*"ukrain" + 0.012*"economi" + 0.011*"gold" + 0.010*"energi" + 0.010*"mariupol" + 0.010*"war" + 0.009*"russia"'),
 (36,
  '0.026*"believ" + 0.022*"elonmusk" + 0.018*"govt" + 0.015*"read" + 0.013*"say" + 0.012*"tru

In [31]:
ldaViz

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
20    -0.084130  0.230255       1        1  3.826807
32    -0.070691 -0.056569       2        1  2.755650
14    -0.078060 -0.177633       3        1  2.664378
16    -0.072741 -0.155548       4        1  2.608988
46    -0.071249  0.016190       5        1  2.493010
31    -0.003200  0.011760       6        1  2.426444
11     0.065776  0.042843       7        1  2.409764
18    -0.001442  0.026240       8        1  2.291165
38    -0.073128 -0.100583       9        1  2.187969
7      0.036284  0.000232      10        1  2.173850
34    -0.056124  0.022933      11        1  2.173117
17     0.047009  0.009871      12        1  2.144780
47     0.177455 -0.081539      13        1  2.101837
26    -0.004979  0.032732      14        1  2.079046
45    -0.013318 -0.004701      15        1  2.065361
15    -0.066094  0.040577      16        1  2.062891
43     0.038005 -0.028231      17        1  2.043420
23     0.160892 -0.017596      18        1  2.029800
35     0.012003  0.027285      19        1  2.026021
19     0.051769  0.040449      20        1  1.987979
21    -0.011896  0.023723      21        1  1.969161
4     -0.040470  0.028998      22        1  1.953679
36    -0.042983  0.019009      23        1  1.926276
28     0.087627  0.038312      24        1  1.921652
9      0.099338  0.075201      25        1  1.908421
3      0.104850 -0.007428      26        1  1.907425
25    -0.018759  0.003516      27        1  1.904548
42    -0.017414  0.052947      28        1  1.885452
40     0.015506  0.026533      29        1  1.873522
1      0.031832  0.019582      30        1  1.871392
29    -0.025519  0.030087      31        1  1.866388
12    -0.055661 -0.070142      32        1  1.848830
0     -0.047630 -0.042685      33        1  1.828311
6     -0.046411 -0.042750      34        1  1.819715
13    -0.013716  0.016948      35        1  1.810652
44    -0.038702  0.007262      36        1  1.808227
5      0.110185  0.005603      37        1  1.793592
33    -0.068017  0.068529      38        1  1.782453
37    -0.014964 -0.042825      39        1  1.775240
2      0.014516 -0.053559      40        1  1.764945
48    -0.001412  0.010203      41        1  1.756801
30    -0.016748  0.010874      42        1  1.735909
22    -0.044303  0.015805      43        1  1.719259
49     0.068662 -0.052892      44        1  1.671960
10     0.092217 -0.055554      45        1  1.629865
27    -0.023553 -0.009307      46        1  1.621119
8     -0.016389  0.021518      47        1  1.620915
24    -0.038802 -0.037142      48        1  1.560678
39    -0.013727  0.021814      49        1  1.554673
41    -0.021695  0.038854      50        1  1.356664, topic_info=            Term        Freq       Total Category  logprob  loglift
14444     marvin  563.000000  563.000000  Default  30.0000  30.0000
14453  marvininu  375.000000  375.000000  Default  29.0000  29.0000
133       follow  277.000000  277.000000  Default  28.0000  28.0000
128        price  438.000000  438.000000  Default  27.0000  27.0000
771       ukrain  313.000000  313.000000  Default  26.0000  26.0000
...          ...         ...         ...      ...      ...      ...
1345       watch    9.092386  169.325939  Topic50  -5.4372   1.3758
255         dont    9.164147  337.181210  Topic50  -5.4294   0.6948
390         mani    8.019936  157.411357  Topic50  -5.5628   1.3232
1919      social    7.562840   97.848623  Topic50  -5.6214   1.7400
725        state    7.378398  248.834233  Topic50  -5.6461   0.7819

[3155 rows x 6 columns], token_table=       Topic      Freq           Term
term                                 
21027     48  0.819206     __annchees
17495     35  0.875336            aaj
16979     20  0.169764  aamaadmiparti
16979     49  0.679054  aamaadmiparti
20654     21  0.775857          aamir
...      ...       ...            ...
8359      31  0.885365         zoltan
12189

In [32]:
def translateLdaIdx(myLdaModel, myLdaViz):
    """Translate lda model topics to match the topics in pyLDAvis visualization"""
    ldaVizIdx = myLdaViz[0].index
    return list(ldaVizIdx)

In [33]:
newIdx = translateLdaIdx(kagLda,ldaViz)


In [34]:
def createDenseMat(myLdaModel,myCorpus,newIdx):
    """Transform corpus to dataframe with topics matching lda visualization"""
    numTopics = myLdaModel.num_topics
    myDense = corpus2dense(myLdaModel[myCorpus],numTopics)
    myDf = pd.DataFrame(myDense)
    mySortedDf = myDf.transpose()
    mySortedDf = myDf.transpose()[newIdx]
    mySortedDf.columns = ['topic' + str(i + 1) for i in range(numTopics)]
    return mySortedDf

In [35]:
kagDf = createDenseMat(kagLda,kagCorpus,newIdx)


In [36]:
def sortByTopicToIdx(cleanedTweetList,mySortedDf,myTopic,myTopicThresh=0.1):
    """Returns an index of tweets surpassing a topic value threshold"""
    myCleanArray = np.array(cleanedTweetList)
    srtIdx = list(mySortedDf[mySortedDf[myTopic]>myTopicThresh].index)
    return srtIdx

def sortTweetsByIdx(cleanedTweetList,srtIdx):
    """Returns sorted tweets as a list based on a defined sort index"""
    myCleanArray = np.array(cleanedTweetList)
    srtTweets = list(myCleanArray[srtIdx])
    return srtTweets

In [37]:
sortedIdx = sortByTopicToIdx(cleanKagTweetList,kagDf,'topic2',myTopicThresh=0.1)


/tmp/ipykernel_52727/43916758.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  myCleanArray = np.array(cleanedTweetList)


In [38]:
fig = plt.figure(figsize=(4,4), dpi=1600)
ax = plt.subplot(111)
plt.hist(kagDf.topic2,bins=20)

ax.tick_params(labelsize=16)
ax.set_xlabel('Bad service topic value',fontsize=24,fontweight='bold')
ax.set_ylabel('Tweets (Topic 2)',fontsize=24,fontweight='bold')
plt.show()
x = fig.tight_layout()

In [39]:
def makeWordCloud(cleanedTweetList,mySortedDf,myTopic,myTopicThresh=0.1):
    """Create word cloud of tweets passing a given threshold for a given topic"""
    sortedIdx = sortByTopicToIdx(cleanedTweetList,mySortedDf,myTopic,myTopicThresh=0.1)
    mySortedTweets = sortTweetsByIdx(cleanedTweetList,sortedIdx)
    filteredWords = ' '.join([' '.join(string) for string in mySortedTweets])
    myTopicCloud = WordCloud(max_font_size=100,scale=8).generate(filteredWords)
    fig = plt.figure(figsize=(10,10), dpi=1600)
    plt.imshow(myTopicCloud)
    plt.axis("off")
    plt.show()

In [40]:
makeWordCloud(cleanKagTweetList,kagDf,'topic2',myTopicThresh=0.2)


/tmp/ipykernel_52727/43916758.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  myCleanArray = np.array(cleanedTweetList)
/tmp/ipykernel_52727/43916758.py:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  myCleanArray = np.array(cleanedTweetList)
